In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

## Exercise

Extract the names, parties and states of [the US Senate](https://www.congress.gov/members?pageSize=100&q={%22congress%22:%22116%22,%22chamber%22:%22Senate%22})

What are the states that have R and D senators?

In [2]:
r = requests.get('https://www.congress.gov/members?pageSize=100&q={%22congress%22:%22116%22,%22chamber%22:%22Senate%22')

In [3]:
r.status_code

200

In [4]:
html = r.text

In [5]:
soup = BeautifulSoup(html, 'html')

In [6]:
# extract members data
members = soup.find_all('li', class_="expanded")

In [18]:
#does not work given inconsistency in the data
def parse_member(member):
    name = member.find("span", class_="result-heading").text    
    # the remaining items are
    remaining_items = member.find_all("span", class_="result-item")
    # extract state and party
    state = remaining_items[0].find("span").text
    party = remaining_items[1].find("span").text
    return {'Name': name, 'State': state, 'Party': party}

In [45]:
def parse_member2(member):
    name = member.find("span", class_="result-heading").text  
    remaining_text = member.text
    party = re.compile("Party:\s+([\w ]+)").findall(remaining_text)[0]
    state = re.compile("State:\s+([\w ]+)").findall(remaining_text)[0]
    return {'Name': name, 'State': state, 'Party': party}

In [46]:
data = [parse_member2(m) for m in members]

In [47]:
df = pd.DataFrame(data)
df.head()

,Name,Party,State
0,"Senator Abdnor, James",Republican,South Dakota
1,"Representative Abercrombie, Neil",Democratic,Hawaii
2,"Senator Abourezk, James",Democratic,South Dakota
3,"Representative Abraham, Ralph Lee",Republican,Louisiana
4,"Senator Abraham, Spencer",Republican,Michigan


In [48]:
# What are the states that have R and D senators?

In [50]:
df2 = df.groupby(["Party", "State"])['Name'].count().unstack("Party").fillna(0)

In [54]:
df2[df2['Democratic'] * df2['Republican'] > 0].index

Index(['Alabama', 'California', 'Florida', 'Georgia', 'Illinois', 'Kentucky',
       'Michigan', 'Minnesota', 'Nebraska', 'North Carolina', 'Ohio',
       'Pennsylvania', 'South Dakota', 'Tennessee', 'Texas'],
      dtype='object', name='State')